In [ ]:
import argparse
import os
import sys
import time, re
import numpy as np
import logging
from datetime import datetime, timedelta
from collections import OrderedDict


username="gupt1075"
# sys.path.append(os.path.dirname(os.path.realpath(__file__)) + '/../')

if (f"/home/{username}/FourCastNet_gil" not in sys.path):
    sys.path.append(f"/home/{username}/FourCastNet_gil")





import h5py
import torch
import torch.cuda.amp as amp
import torch.distributed as dist
import torch.nn as nn
import torchvision
from numpy.core.numeric import False_
from torch.nn.parallel import DistributedDataParallel
from torchvision.utils import save_image
from utils import logging_utils
from utils.weighted_acc_rmse import (
    unweighted_acc_torch_channels,
    weighted_acc_masked_torch_channels,
    weighted_acc_torch_channels,
    weighted_rmse_torch_channels,
)

import glob
from datetime import datetime, timedelta

import matplotlib.pyplot as plt
import wandb
from networks.afnonet import AFNONet
from utils.data_loader_multifiles import get_data_loader
from utils.YParams import YParams
from scipy.stats import sem



In [ ]:

# ensemble_output_date_01_February_2018_ensemble_number_4
# Set the datetime format to search for in the filenames
datetime_format = '%Y_%B_%d_%H'

# Set the start and end dates to search for
start_date = datetime(2018, 1, 31, 0, 0, 0)
end_date = datetime(2018, 2, 7, 23, 59, 59)
directory = "/scratch/gilbreth/gupt1075/ERA5_ensemble_feb_1/afno_backbone/222/"
# Loop through all files in the directory
for filename in os.listdir(directory):
    # Check if the file is a NumPy file
    if filename.endswith('.npy'):
        # Try to extract the datetime from the filename
        try:
            file_datetime = datetime.strptime(filename.split('.')[0], datetime_format)
            # Check if the file datetime is within the specified range
            if start_date <= file_datetime <= end_date:
                # Load the NumPy file
                file_path = os.path.join(directory, filename)
                data = np.load(file_path)
                print(f'Loaded file: {filename}')
                # Do something with the data
                print(data.shape)
        except ValueError:
            # If the filename doesn't match the expected format, skip the file
            pass